# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, hamming_loss
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#nltk for tokenizing/lemmatizing
import nltk
nltk.download(['punkt','wordnet'])



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///textmessages.db')
df = pd.read_sql_table('MessageTable',con=engine)
df_orig_rows = df.shape[0]

In [3]:
df_orig_rows

26345

In [4]:
df[df.isnull().any(axis=1)]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7465,8365,NOTES: It mark as not enough information,None,direct,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9936,11186,My thoughts and prayers go out to all the live...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9937,11188,I m sorry for the poor people in Haiti tonight...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9938,11189,RT selenagomez UNICEF has just announced an em...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9939,11192,lilithia yes 5.2 magnitude earthquake hit mani...,None,social,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9940,11193,RT TheNewsBlotter RT caribnews On Call Interna...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9941,11195,Most Eureka homeowners won&#39 t collect on qu...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9942,11196,Haiti hit by largest earthquake in over 200 ye...,None,social,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9943,11197,RT pinkelephantpun Earthquake Relief Donate ht...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9944,11198,Praying that today s earthquake in #Haiti mini...,None,social,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
X = df['message']
y = df.iloc[:,4:]

In [6]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y.shape

(26345, 36)

In [9]:
X.shape

(26345,)

#### Observation
'Related' category has 3 values, when these are supposed to be binary. Might eliminate rows that fall out of the binary classification. 'Child-alone' only has 1 value, meaning it won't provide much of anything to the ML process. Might eliminate as well.

In [10]:
#eliminate rows with null values
many_nans = y[y.isnull().any(axis=1)].index

y.drop(many_nans, inplace=True)
X.drop(many_nans, inplace=True)

In [11]:
print(y.shape, X.shape)

(26207, 36) (26207,)


In [12]:
#Further investigation of 'related' column
y['related'].unique()

array([ 1.,  0.,  2.])

In [13]:
y[y['related']==2.].shape

(192, 36)

In [14]:
#eliminate rows where 'related' category is 2 (non-binary)
non_binary_related = y[y['related']==2.].index

y.drop(non_binary_related, inplace=True)
X.drop(non_binary_related, inplace=True)

In [15]:
print(y.shape, X.shape)

(26015, 36) (26015,)


In [16]:
#percentage of data points eliminated
print(f'{round((df_orig_rows - y.shape[0]) / df_orig_rows * 100,2)}% removed')

1.25% removed


### 2. Write a tokenization function to process your text data

In [17]:
#this function will be used later as the tokenizer param in the CountVectorizer 
def tokenize(text):
    #split into tokens
    tokens = nltk.word_tokenize(text)
    
    #get to the ROOT of the words (lemmatize)
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token, pos='v').lower()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [94]:
#pipeline
pipeline = Pipeline(
    [('vectorizer',CountVectorizer(tokenizer=tokenize)),
     ('tfidf',TfidfTransformer()),
     ('clf',RandomForestClassifier(random_state=42))]
)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=42)

In [66]:
y_train.shape

(19511, 36)

In [19]:
for col in y_train:
    uniques = np.unique(y_train[col],return_counts=True)
    print(col, uniques[1])

related [ 4557 14954]
request [16154  3357]
offer [19412    99]
aid_related [11362  8149]
medical_help [17937  1574]
medical_products [18521   990]
search_and_rescue [18975   536]
security [19166   345]
military [18855   656]
child_alone [19511]
water [18255  1256]
food [17319  2192]
shelter [17779  1732]
clothing [19197   314]
money [19062   449]
missing_people [19274   237]
refugees [18851   660]
death [18620   891]
other_aid [16914  2597]
infrastructure_related [18277  1234]
transport [18627   884]
buildings [18520   991]
electricity [19103   408]
tools [19386   125]
hospitals [19304   207]
shops [19427    84]
aid_centers [19286   225]
other_infrastructure [18677   834]
weather_related [14035  5476]
floods [17902  1609]
storm [17685  1826]
fire [19301   210]
earthquake [17655  1856]
cold [19106   405]
other_weather [18467  1044]
direct_report [15698  3813]


In [ ]:
# %time pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### Observation
The classification_report below is with n_estimators = 10

In [68]:
y_pred = pipeline.predict(X_test)
target_names = y.columns
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.78      0.89      0.83      4960
               request       0.42      0.07      0.12      1123
                 offer       0.00      0.00      0.00        20
           aid_related       0.47      0.15      0.23      2729
          medical_help       0.50      0.00      0.00       513
      medical_products       0.00      0.00      0.00       324
     search_and_rescue       0.00      0.00      0.00       188
              security       0.00      0.00      0.00       126
              military       0.25      0.00      0.01       204
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       418
                  food       0.38      0.00      0.01       738
               shelter       1.00      0.00      0.00       587
              clothing       0.00      0.00      0.00        92
                 money       0.00      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [70]:
pipeline.get_params()

{'clf': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
             oob_score=False, random_state=42, verbose=0, warm_start=False),
 'clf__bootstrap': True,
 'clf__class_weight': None,
 'clf__criterion': 'gini',
 'clf__max_depth': None,
 'clf__max_features': 'auto',
 'clf__max_leaf_nodes': None,
 'clf__min_impurity_decrease': 0.0,
 'clf__min_impurity_split': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 2,
 'clf__min_weight_fraction_leaf': 0.0,
 'clf__n_estimators': 10,
 'clf__n_jobs': 1,
 'clf__oob_score': False,
 'clf__random_state': 42,
 'clf__verbose': 0,
 'clf__warm_start': False,
 'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_er

In [71]:
parameters = {'clf__n_estimators':[10,50,100]}
# parameters = {'clf__estimator__n_estimators':[10,50,100], 'vectorizer__stop_words': ['english', None]}

cv = GridSearchCV(pipeline, param_grid = parameters)

In [73]:
cv.fit(X_train, y_train)

In [27]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

In [49]:
pipeline.set_params(clf__n_estimators = 20)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [50]:
%time pipeline.fit(X_train, y_train)

Wall time: 3min 26s


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [75]:
pipeline.set_params(clf__n_estimators=40)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...stimators=40, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [76]:
%time pipeline.fit(X_train, y_train)

Wall time: 1min 29s


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...stimators=40, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

### Observation on pipeline params
It looks like, the more estimators used for the random forest classifier, the more time fitting takes. This makes sense. Will need to balance computing time and results.

In [54]:
parameters = {'clf__estimator__n_estimators':[10,20],'vectorizer__stop_words': ['english', None]}

cv = GridSearchCV(pipeline, param_grid = parameters)
%time cv.fit(X_train, y_train)

Wall time: 24min 17s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'vectorizer__stop_words': ['english', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
#with n_jobs = -1
parameters = {'clf__estimator__n_estimators':[10,20],'vectorizer__stop_words': ['english', None]}

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs=-1)
%time cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

## best params:

clf__estimator_n_estimators: 100

'vectorizer__stop_words': None

In [57]:
pipeline.set_params(clf__estimator__n_estimators=100)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [58]:
%time pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [77]:
y_pred = pipeline.predict(X_test)
target_names = y.columns
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.77      0.97      0.86      4960
               request       0.51      0.03      0.07      1123
                 offer       0.00      0.00      0.00        20
           aid_related       0.48      0.09      0.16      2729
          medical_help       0.50      0.00      0.00       513
      medical_products       0.50      0.00      0.01       324
     search_and_rescue       0.00      0.00      0.00       188
              security       0.00      0.00      0.00       126
              military       0.00      0.00      0.00       204
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       418
                  food       0.33      0.00      0.01       738
               shelter       0.00      0.00      0.00       587
              clothing       1.00      0.01      0.02        92
                 money       0.00      

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [78]:
hamming_loss(y_test,y_pred)

0.07413813721470548

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
#update pipeline
pipeline = Pipeline(
    [('vectorizer',CountVectorizer(tokenizer=tokenize)),
     ('tfidf',TfidfTransformer()),
     ('svd',TruncatedSVD()),
     ('clf',RandomForestClassifier(random_state=42, n_estimators=100))]
)

In [28]:
pipeline.set_params(svd__n_components=100)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...timators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [29]:
%time pipeline.fit(X_train,y_train)

Wall time: 3min 49s


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...timators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [30]:
y_pred = pipeline.predict(X_test)
hamming_loss(y_test,y_pred)

0.07489835315019817

In [100]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vectorizer', 'tfidf', 'svd', 'clf', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__preprocessor', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'svd__algorithm', 'svd__n_components', 'svd__n_iter', 'svd__random_state', 'svd__tol', 'clf__bootstrap', 'clf__class_weight', 'clf__criterion', 'clf__max_depth', 'clf__max_features', 'clf__max_leaf_nodes', 'clf__min_impurity_decrease', 'clf__min_impurity_split', 'clf__min_samples_leaf', 'clf__min_samples_split', 'clf__min_weight_fraction_leaf', 'clf__n_estimators', 'clf__n_jobs', 'clf__oob_score', 'clf__random_state', 'clf__v

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.